# Video game sales and ratings dataset - Kaggle
---

Kaggle link: https://www.kaggle.com/kendallgillies/video-game-sales-and-ratings

---
### Import the dataset
---

In [1]:
import pandas as pd

In [2]:
games_df = pd.read_csv('Video_Game_Sales_as_of_Jan_2017.csv')

print(games_df.shape)
games_df.head()

(17416, 15)


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.54,76.0,51.0,8.0,324.0,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.80,3.79,3.29,35.57,82.0,73.0,8.3,712.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.95,3.28,2.95,32.78,80.0,73.0,8.0,193.0,E
4,Pokemon Red/Pokemon Blue,G,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN


Not obvious here but sales are in millions of copies.

---
### Missing values
---

In [3]:
vars_with_na = [
    var for var in games_df.columns
    if games_df[var].isnull().sum() > 0
]

vars_with_na

['Year_of_Release',
 'Publisher',
 'Critic_Score',
 'Critic_Count',
 'User_Score',
 'User_Count',
 'Rating']

In [4]:
round(games_df[vars_with_na].isnull().mean(),4)

Year_of_Release    0.0005
Publisher          0.0001
Critic_Score       0.5214
Critic_Count       0.5214
User_Score         0.5523
User_Count         0.5523
Rating             0.4113
dtype: float64

We now have a list of all features with null values and their percentage of null values to 4 significant figures.\
The critic, user and rating features all have high proportions of null values.\
I could possibly drop the year_of_release NaN values and not lose too much information

---
### Unique values
---

In [5]:
print(len(games_df.columns))
games_df.nunique()

15


Name               12080
Platform              31
Year_of_Release       42
Genre                 12
Publisher            627
NA_Sales             399
EU_Sales             306
JP_Sales             245
Other_Sales          157
Global_Sales         627
Critic_Score          82
Critic_Count         106
User_Score            95
User_Count           903
Rating                 8
dtype: int64

A list of the number of unique values in each feature - NaN values are excluded from the count.\
Though this is ok, separately viewing the unique vars for numerical and categorical features would be better.

In [6]:
num_vars = [
    var for var in games_df.columns
    if games_df[var].dtype != 'O'
]

print(len(num_vars))
num_vars

10


['Year_of_Release',
 'NA_Sales',
 'EU_Sales',
 'JP_Sales',
 'Other_Sales',
 'Global_Sales',
 'Critic_Score',
 'Critic_Count',
 'User_Score',
 'User_Count']

In [7]:
cat_vars = [
    var for var in games_df.columns
    if var not in num_vars
]

print(len(cat_vars))
cat_vars

5


['Name', 'Platform', 'Genre', 'Publisher', 'Rating']

In [8]:
games_df[num_vars].nunique()

Year_of_Release     42
NA_Sales           399
EU_Sales           306
JP_Sales           245
Other_Sales        157
Global_Sales       627
Critic_Score        82
Critic_Count       106
User_Score          95
User_Count         903
dtype: int64

In [9]:
games_df[cat_vars].nunique()

Name         12080
Platform        31
Genre           12
Publisher      627
Rating           8
dtype: int64

Nothing seems off with the unique values, before moving on to the pre-processing stage I would like to plot some graphs to see the relationship between the variables and the global sales.

---
### Pre-processing
---

##### Drop null values from year of release and Publisher
---

In [10]:
vars_with_na

['Year_of_Release',
 'Publisher',
 'Critic_Score',
 'Critic_Count',
 'User_Score',
 'User_Count',
 'Rating']

In [11]:
games_df = games_df.dropna(subset=['Year_of_Release', 'Publisher'])

In [12]:
games_df.shape

(17408, 15)

8 rows/games have been removed - the original shape was (17416, 15).

In [13]:
vars_with_na = [
    var for var in games_df.columns
    if games_df[var].isnull().sum() > 0
]

vars_with_na

['Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Rating']

In [14]:
games_df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.54,76.0,51.0,8.0,324.0,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.80,3.79,3.29,35.57,82.0,73.0,8.3,712.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.95,3.28,2.95,32.78,80.0,73.0,8.0,193.0,E
4,Pokemon Red/Pokemon Blue,G,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN


##### Convert year of release to int
---

It does not look right to have a year written with decimals.

In [15]:
games_df['Year_of_Release'] = games_df['Year_of_Release'].astype('int32')

In [16]:
games_df.dtypes

Name                object
Platform            object
Year_of_Release      int32
Genre               object
Publisher           object
NA_Sales           float64
EU_Sales           float64
JP_Sales           float64
Other_Sales        float64
Global_Sales       float64
Critic_Score       float64
Critic_Count       float64
User_Score         float64
User_Count         float64
Rating              object
dtype: object

Now the years have no decimal places.\
This could have been fixed in tableau as well.

##### Add a new column with the full names of the platforms for clearer labelling on tableau
---

Some of these game platforms are extremely old and many 'younger' people won't know what they mean, so we need a column with their full names.

In [17]:
platform_list = games_df['Platform'].unique().tolist()

In [18]:
print(len(platform_list))
platform_list

31


['Wii',
 'NES',
 'G',
 'DS',
 'X360',
 'PS3',
 'PS2',
 'SNES',
 'GBA',
 '3DS',
 'PS4',
 'N64',
 'PS',
 'X',
 'PC',
 '2600',
 'PSP',
 'XOne',
 'WiiU',
 'GC',
 'GEN',
 'DC',
 'PSV',
 'SAT',
 'SCD',
 'WS',
 'NG',
 'TG16',
 '3DO',
 'GG',
 'PCFX']

In [19]:
ext_platform_list = [
    'Nintendo Wii', 'Nintendo Entertainment System', 'Game Boy', 'Nintendo DS', 'Xbox 360',
    'Play Station 3', 'Play Station 2', 'Super Nintendo Entertainment System', 'Game Boy Advance', 'Nintendo 3DS',
    'Play Station 4', 'Nintendo 64', 'Play Station', 'Xbox', 'PC',
    'Atari 2600', 'Play Station Portable', 'Xbox One', 'Nintendo Wii U', 'GameCube',
    'Sega Genesis', 'Sega Dreamcast', 'Play Station Vita', 'Sega Saturn', 'Sega CD',
    'WonderSwan', 'Neo Geo', 'TruboGrafx-16', '3DO Interactive Multiplayer', 'Game Gear',
    'PC-FX'
]
print(len(ext_platform_list))

31


In [20]:
def i_dont_know_the_platform(df, search_word, col='Platform'):
    return df.loc[df[col] == search_word]

#created for me to output rows of platform abbreviations i didn't understand.
#game name, year of release, and platform abbreviation were then googled.

In [21]:
i_dont_know_the_platform(games_df, 'GG')

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
13639,Sonic the Hedgehog 2 (8-bit),GG,1992,Platform,Sega,0.0,0.0,0.04,0.0,0.04,NaN,NaN,NaN,NaN,NaN


In [22]:
#zip both the lists
zipped = zip(platform_list, ext_platform_list)

In [23]:
#create the dictionary
platform_dict = dict(zipped)

#test the dict
platform_dict['GG']

'Game Gear'

In [24]:
ext_platform_col = []

for i in games_df['Platform']:
    ext_platform_col.append(platform_dict[i])

ext_platform_col

['Nintendo Wii',
 'Nintendo Entertainment System',
 'Nintendo Wii',
 'Nintendo Wii',
 'Game Boy',
 'Game Boy',
 'Nintendo DS',
 'Nintendo Wii',
 'Nintendo Wii',
 'Nintendo Entertainment System',
 'Nintendo DS',
 'Nintendo DS',
 'Game Boy',
 'Nintendo Wii',
 'Xbox 360',
 'Nintendo Wii',
 'Play Station 3',
 'Play Station 2',
 'Super Nintendo Entertainment System',
 'Nintendo DS',
 'Nintendo DS',
 'Game Boy',
 'Nintendo Entertainment System',
 'Xbox 360',
 'Play Station 2',
 'Game Boy Advance',
 'Nintendo DS',
 'Nintendo DS',
 'Play Station 2',
 'Nintendo 3DS',
 'Play Station 4',
 'Xbox 360',
 'Xbox 360',
 'Game Boy',
 'Play Station 3',
 'Xbox 360',
 'Xbox 360',
 'Play Station 4',
 'Play Station 3',
 'Play Station 2',
 'Nintendo 3DS',
 'Nintendo Wii',
 'Play Station 3',
 'Nintendo DS',
 'Xbox 360',
 'Nintendo 3DS',
 'Nintendo 64',
 'Nintendo DS',
 'Play Station 2',
 'Nintendo 3DS',
 'Nintendo Wii',
 'Game Boy',
 'Xbox 360',
 'Play Station',
 'Nintendo 3DS',
 'Play Station 3',
 'Play Stati

In [25]:
games_df['Extended_Platform_Name'] = ext_platform_col

In [26]:
games_df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating,Extended_Platform_Name
0,Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.54,76.0,51.0,8.0,324.0,E,Nintendo Wii
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,Nintendo Entertainment System
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.80,3.79,3.29,35.57,82.0,73.0,8.3,712.0,E,Nintendo Wii
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.95,3.28,2.95,32.78,80.0,73.0,8.0,193.0,E,Nintendo Wii
4,Pokemon Red/Pokemon Blue,G,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,Game Boy


---
### Save new csv for tableau
---

In [27]:
games_df.to_csv('for_tableau_games_sales_till_jan_2017.csv')